# 🤖**RAG Chatbot**

In [2]:
# ========================================
# Imports: Core LangChain and Dependencies
# ========================================

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma as ChromaCommunity

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

import os
from dotenv import load_dotenv

# ========================
# Environment Preparation
# ========================

load_dotenv()  # Load environment variables from a .env file

# =============================
# Step 1: Load HTML Document(s)
# =============================

loader = UnstructuredHTMLLoader(file_path="data/mg-zs-warning-messages.html")
car_docs = loader.load()

# Optional: Preview first loaded document
# print(car_docs[0])

# =============================
# Step 2: Split HTML into Chunks
# =============================

chunk_size = 300
chunk_overlap = 100

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", " ", ""]
)

docs = splitter.split_documents(car_docs)

# Optional: Preview first chunk
print(docs[0])

# =============================
# Step 3: Generate Embeddings
# =============================

embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Store embeddings in a persistent Chroma vector store
vectorstore = Chroma.from_documents(
    docs,
    embedding=embedding_function,
    persist_directory=os.getcwd()
)

# ===============================
# Step 4: Configure the Retriever
# ===============================

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# ==========================
# Step 5: Define the Prompt
# ==========================

prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="""
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}
Context: {context}
Answer:"""
)

# ==========================
# Step 6: Initialize the LLM
# ==========================

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.7,
    max_tokens=100
)

# =====================================
# Step 7: Create the Retrieval QA Chain
# =====================================

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
)

# ======================================
# Step 8: Query the System with a Prompt
# ======================================

question = "The Gasoline Particular Filter Full warning has appeared. What does this mean and what should I do about it?"
response = rag_chain.invoke(question)

# Print the final answer
print(response.content)

page_content='Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer' metadata={'source': 'data/mg-zs-warning-messages.html'}
The "Gasoline Particular Filter Full" warning indicates that the gasoline particular filter is full and needs to be replaced. You should consult an MG Authorised Repairer as soon as possible to have it replaced. They can properly diagnose and fix the issue.
